In [3]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [4]:
from src.data_preprocessing import DataLoaders

loaders = DataLoaders(trial_data=True)
train_dataloader = loaders.get_train_dataloader()
val_dataloader = loaders.get_val_dataloader()

In [25]:
from src.models.tiny_vit_lightning import TinyVitLightning
from lightning import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import json
from lightning.pytorch.loggers import WandbLogger


with open('../data/country_to_index_mapped.json', 'r') as f:
    num_classes = len(json.load(f))

model = TinyVitLightning(num_classes)
wandb_logger = WandbLogger()
trainer = Trainer(max_epochs=3, callbacks=[EarlyStopping(monitor="val_loss", mode="min")], logger=wandb_logger)
trainer.fit(model, train_dataloader, val_dataloader)


ValueError: `Trainer(barebones=True, logger=<lightning.pytorch.loggers.wandb.WandbLogger object at 0x17832df90>)` was passed. Logging can impact raw speed so it is disabled in barebones mode.

In [24]:
import torch
from huggingface_hub import Repository
import os
import shutil

model_save_path = "../data/models/"
repo = Repository(local_dir=model_save_path, clone_from="mwitjez/geoguessr_tiny_ViT")

input_sample = torch.randn(1, 3, 224, 224)
model.to_onnx(f"{model_save_path}/model.onxx", input_sample, export_params=True)
torch.save(model.state_dict(), f"{model_save_path}/geoguessr_model.bin")

repo.push_to_hub()

if os.path.exists(model_save_path):
    shutil.rmtree(model_save_path)
    print(f"'{model_save_path}' has been removed successfully.")
else:
    print(f"'{model_save_path}' does not exist.")


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/mwitjez/geoguessr_tiny_ViT into local empty directory.


Download file geoguessr_model.bin:   0%|          | 8.74k/79.0M [00:00<?, ?B/s]

Download file geoguessr_model_state_dict.bin:   0%|          | 8.00k/79.0M [00:00<?, ?B/s]

Download file .DS_Store: 100%|##########| 6.00k/6.00k [00:00<?, ?B/s]

Clean file .DS_Store: 100%|##########| 6.00k/6.00k [00:00<?, ?B/s]

/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:340: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert L == H * W, "input feature has wrong size"
/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:140: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  B = len(x)
Adding files tracked by Git LFS: ['model.onxx']. This may take a bit of time if the files are large.


Upload file model.onxx:   0%|          | 1.00/89.9M [00:00<?, ?B/s]

Upload file geoguessr_model.bin:   0%|          | 1.00/79.0M [00:00<?, ?B/s]

To https://huggingface.co/mwitjez/geoguessr_tiny_ViT
   8acb880..db4ed3f  main -> main



'../data/models/' has been removed successfully.
